# setup

In [1]:
import os

# isso aqui é so sair da pasta de reproducao
if os.getcwd().split('/')[-1] == 'reproducao' or os.getcwd().split('\\')[-1] == 'reproducao':
    os.chdir('..')
    print(os.getcwd())

c:\Users\Pichau\Documents\git\lesaude\strodthoff2024prospects\src


In [2]:
import wfdb
import os
import shutil
import zipfile
import numpy as np
import pandas as pd
import resampy
import datetime
import h5py

from tqdm.auto import tqdm
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

In [3]:
from reproducao_timeseries_utils import *

In [4]:
channel_stoi_default = {"i": 0, "ii": 1, "v1":2, "v2":3, "v3":4, "v4":5, "v5":6, "v6":7, "iii":8, "avr":9, "avl":10, "avf":11, "vx":12, "vy":13, "vz":14}

# utils

In [5]:
def get_stratified_kfolds(labels,n_splits,random_state):
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    return skf.split(np.zeros(len(labels)),labels)

In [6]:
def resample_data(sigbufs, channel_labels, fs, target_fs, channels=12, channel_stoi=None):#,skimage_transform=True,interpolation_order=3):
    channel_labels = [c.lower() for c in channel_labels]
    #https://github.com/scipy/scipy/issues/7324 zoom issues
    factor = target_fs/fs
    timesteps_new = int(len(sigbufs)*factor)
    if(channel_stoi is not None):
        data = np.zeros((timesteps_new, channels), dtype=np.float32)
        for i,cl in enumerate(channel_labels):
            if(cl in channel_stoi.keys() and channel_stoi[cl]<channels):
                data[:,channel_stoi[cl]] = resampy.resample(sigbufs[:,i], fs, target_fs).astype(np.float32)
    else:
        data = resampy.resample(sigbufs, fs, target_fs, axis=0).astype(np.float32)
    return data

In [7]:
def fix_nans_and_clip(signal,clip_amp=3):
    for i in range(signal.shape[1]):
        tmp = pd.DataFrame(signal[:,i]).interpolate().values.ravel().tolist()
        signal[:,i]= np.clip(tmp,a_max=clip_amp, a_min=-clip_amp) if clip_amp>0 else tmp

# prepare_mimicecg

In [8]:
data_path = 'D:\datasets\MIMIC-IV-ECG-DEMO\mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0'
clip_amp=3
target_fs=100
channels=12
strat_folds=20
channel_stoi=channel_stoi_default
target_folder = 'D:\datasets\MIMIC-IV-ECG-DEMO'
recreate_data=True

In [9]:
target_folder = Path(target_folder)
target_folder.mkdir(parents=True, exist_ok=True)
target_folder

WindowsPath('D:/datasets/MIMIC-IV-ECG-DEMO')

In [10]:
# zip_path = 'D:\datasets\MIMIC-IV-ECG-DEMO\mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip'
# with zipfile.ZipFile(Path(zip_path), 'r') as archive:
#     lst = archive.namelist()
#     lst = [x for x in lst if x.endswith(".hea")]
# len(lst), lst

In [11]:
lst = []
for root, dirs, files in os.walk(data_path):
    relative = os.path.relpath(root, data_path)
    for file in files:
        lst.append(os.path.join(relative, file))
lst = [x for x in lst if x.endswith(".hea")]
len(lst), lst

(795,
 ['files\\p1000\\p10000032\\s40689238\\40689238.hea',
  'files\\p1000\\p10000032\\s44458630\\44458630.hea',
  'files\\p1000\\p10000032\\s49036311\\49036311.hea',
  'files\\p1000\\p10000117\\s45090959\\45090959.hea',
  'files\\p1000\\p10000117\\s48446569\\48446569.hea',
  'files\\p1000\\p10000285\\s42709053\\42709053.hea',
  'files\\p1000\\p10000560\\s41445586\\41445586.hea',
  'files\\p1000\\p10000560\\s42695383\\42695383.hea',
  'files\\p1000\\p10000635\\s40067704\\40067704.hea',
  'files\\p1000\\p10000635\\s42947358\\42947358.hea',
  'files\\p1000\\p10000635\\s43522917\\43522917.hea',
  'files\\p1000\\p10000635\\s44095784\\44095784.hea',
  'files\\p1000\\p10000635\\s45386375\\45386375.hea',
  'files\\p1000\\p10000635\\s48339811\\48339811.hea',
  'files\\p1000\\p10000764\\s40539087\\40539087.hea',
  'files\\p1000\\p10000764\\s43681375\\43681375.hea',
  'files\\p1000\\p10000764\\s44069449\\44069449.hea',
  'files\\p1000\\p10000764\\s44837313\\44837313.hea',
  'files\\p1000\\p1000

In [12]:
size = len(lst)
length = 1000
leads = 12

output_hdf5 = h5py.File(os.path.join(target_folder, 'MIMICstrodthoff.h5'), 'w')
x = output_hdf5.create_dataset('tracings', shape = (size, length, leads), dtype = np.float32)
study_id = output_hdf5.create_dataset('study_id', shape = (size,), dtype = np.int64)
subject_id = output_hdf5.create_dataset('subject_id', shape = (size,), dtype = np.int64)

In [13]:
meta = []
for idx, l in tqdm(enumerate(lst)):
    # archive.extract(l, path="tmp_dir/")
    # archive.extract(l[:-3]+"dat", path="tmp_dir/")
    filename = Path(data_path)/l
    sigbufs, header = wfdb.rdsamp(str(filename)[:-4])

    tmp={}
    tmp["data"]=filename.parent.parent.stem+"_"+filename.parent.stem # subject_id + patientid_study
    tmp["study_id"]=int(filename.stem)
    tmp["subject_id"]=int(filename.parent.parent.stem[1:])
    tmp['ecg_time']= datetime.datetime.combine(header["base_date"],header["base_time"])
    tmp["nans"]= list(np.sum(np.isnan(sigbufs),axis=0))#save nans channel-dependent
    if(np.sum(tmp["nans"])>0):#fix nans
        fix_nans_and_clip(sigbufs,clip_amp=clip_amp)
    elif(clip_amp>0):
        sigbufs = np.clip(sigbufs,a_max=clip_amp,a_min=-clip_amp)

    data = resample_data(sigbufs=sigbufs,channel_stoi=channel_stoi,channel_labels=header['sig_name'],fs=header['fs'],target_fs=target_fs,channels=channels)
    
    assert(target_fs<=header['fs'])
    meta.append(tmp)
    x[idx, :, :] = data
    study_id[idx] = tmp['study_id']
    subject_id[idx] = tmp['subject_id']

0it [00:00, ?it/s]

In [15]:
df = pd.DataFrame(meta)

dataset_add_mean_col(df,output_hdf5, data_folder=target_folder)
dataset_add_std_col(df,output_hdf5, data_folder=target_folder)
dataset_add_length_col(df,output_hdf5, data_folder=target_folder)

#save means and stds
mean, std = dataset_get_stats(df)

#save
lbl_itos=[]
save_dataset(df,lbl_itos,mean,std,target_folder)

In [16]:
output_hdf5.close()